In [ ]:
import numpy as np
import os
import sys
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
sys.path.append('../')
import utils.csv as csv
import utils.shp as shp

In [ ]:
PATH='D:\\Deutschland\\FUB\\master_thesis\\data'
DATA_DIR = os.path.join(PATH, 'gee', 'output', 'validation_grid_daily_padding')
LABEL_CSV = 'label_val_grid.csv'
METHOD = 'classification'
MODEL = 'lstm'
UID = '8pure9'
MODEL_NAME = MODEL + '_' + UID
LABEL_PATH = os.path.join(PATH, 'ref', 'validation', LABEL_CSV)
MODEL_PATH = f'../outputs/models/{METHOD}/01/{MODEL_NAME}.pth'

In [ ]:
x_data, y_data = csv.to_numpy(DATA_DIR, LABEL_PATH)

In [ ]:
predicted = np.random.randint(0, 9, 17772)
y_data[:, 1] = predicted
y_list = y_data.tolist()
cols = ['id', 'class']
pred = csv.list_to_dataframe(y_list, cols)


In [ ]:
in_path = 'D:\\Deutschland\\FUB\\master_thesis\\data\\Reference_data\\validation\\wgs_validation_grid_20m.shp'
gdf = shp.load_shp_file(in_path)

In [ ]:
gdf.rename(columns={'id_1':'id'}, inplace=True)
gdf

In [ ]:
polygons = pd.merge(gdf, pred, on='id', how='inner')
polygons

In [ ]:
class_name = {0:'Spruce', 1:'Sliver Fir', 2:'Douglas Fir', 3:'Pine', 
            4:'Oak', 5:'Red Oak', 6:'Beech', 7:'Sycamore', 8:'Others'}
polygons['name'] = polygons['class'].map(class_name)
polygons

In [ ]:
# top area: bottom > 2850000
sub_area1 = polygons[polygons['top'] > 2850000]
# middle area: 2800000 < bottom < 2850000
sub_area2 = polygons[(polygons['top'] < 2850000) & (polygons['top'] > 2800000)]
# bottoem area: bottom < 2800000
sub_area3 = polygons[polygons['top'] < 2800000]

In [ ]:
# Create a figure and axis
fig, ax = plt.subplots(figsize=(20, 6))

# Plot all the polygons
sub_area1.plot(ax=ax, column='name', legend=True, categorical = True, legend_kwds={'loc': 'center left', 'bbox_to_anchor':(1,0.5)})

# Set the title and axis labels
ax.set_title("Tree Species Map")
ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")
# Show the plot
plt.show()